In [3]:
# imports for training
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
# import dataset, network to train and metric to optimize
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss


In [18]:
import pandas as pd
import numpy as np
data=pd.read_csv('./data/ETH_histical_data.csv')

In [19]:
data.date=pd.DatetimeIndex(data.date)
data['date_seconds']=data.date.astype(np.int64) // 10**9

/opt/anaconda3/envs/python3_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [20]:
data['crypto']='ETH'
data.fillna(mathod='ffill', )

In [23]:
# define the dataset, i.e. add metadata to pandas dataframe for the model to understand it
max_encoder_length = 36
max_prediction_length = 6
training_cutoff = "2021-06-01"  # day for cutoff

training = TimeSeriesDataSet(
    data[lambda x: x.date <= training_cutoff],
    time_idx= 'date_seconds',  # column name of time of observation
    target='close',  # column name of target to predict
    group_ids=[ 'crypto' ],  # column name(s) for timeseries IDs
    max_encoder_length=max_encoder_length,  # how much history to use
    max_prediction_length=max_prediction_length,  # how far to predict into future
    # covariates static for a timeseries ID
    #static_categoricals=[ ... ],
    #static_reals=[ ... ],
    # covariates known and unknown in the future to inform prediction
    #time_varying_known_categoricals=[  ],
    #time_varying_known_reals=[ ... ],
    #time_varying_unknown_categoricals=[ 'volume' ],
    time_varying_unknown_reals=['close','volume' ],
    allow_missing_timesteps=True
)


/opt/anaconda3/envs/python3_env/lib/python3.7/site-packages/pytorch_forecasting/data/timeseries.py:1244: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__crypto': 'ETH'}]
  UserWarning,


AssertionError: filters should not remove entries all entries - check encoder/decoder lengths and lags

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     721 non-null    datetime64[ns]
 1   open                     720 non-null    float64       
 2   close                    721 non-null    float64       
 3   high                     720 non-null    float64       
 4   low                      720 non-null    float64       
 5   volume                   720 non-null    float64       
 6   market_cap               702 non-null    float64       
 7   url_shares               716 non-null    float64       
 8   unique_url_shares        716 non-null    float64       
 9   reddit_posts             710 non-null    float64       
 10  reddit_posts_score       710 non-null    float64       
 11  reddit_comments          710 non-null    float64       
 12  reddit_comments_score    710 non-nul